In [1]:
!pip install opencv-python
!pip install numpy
!pip install keras

In [1]:
import cv2
import numpy as np
from keras.models import load_model

In [2]:
model=load_model(".\\")

In [3]:
labels_dict={0:'without mask',1:'mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
webcam = cv2.VideoCapture(0) #Use camera 0

In [4]:
# We load the xml file
classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [8]:
while True:
    (rval, im) = webcam.read()
# for im in ls:
#     name = im
#     path = os.path.join(iPath, "without_mask", im)
#     im = cv2.imread(path)
    im=cv2.flip(im,1,1) #Flip to act as a mirror
    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        #print(result)

        label=np.argmax(result,axis=1)[0]

        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
#         print(label)
    # Show the image
    cv2.imshow("live", im)
#     cv2.imwrite(os.path.join(os.getcwd(), "t2", name), im)
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
    # Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()